In [1]:
import os
import pandas as pd
import ast

# Specify the file path
data_file = os.path.join("datatest", "filtered", "filtered_tags.csv")

if not os.path.exists(data_file):
    print(f"File not found: {data_file}")
else:
    # Read the CSV file
    data = pd.read_csv(data_file)
    df = pd.DataFrame(data)

    if 'tags' in df.columns:
        # Convert the tags from string to list and flatten them
        all_tags = df['tags'].apply(lambda x: ast.literal_eval(x)).explode()
        # Get unique tags
        unique_tags = all_tags.unique()

        # Create the new folder if it doesn't exist
        output_dir = os.path.join("great_data")
        os.makedirs(output_dir, exist_ok=True)

        # Save the unique tags to a file
        tags_file = os.path.join(output_dir, "tags.txt")
        with open(tags_file, 'w') as f:
            for tag in unique_tags:
                f.write(f"{tag}\n")

        print(f"Unique tags saved to: {tags_file}")
    else:
        print("'tags' column not found in the CSV file.")


File not found: datatest\filtered\filtered_tags.csv
